In [2]:
import pandas as pd
import numpy as np

In [12]:
# macronutrients(protien,fat,carbohydarte) for diffent body goals
weight = float(input("Enter your weight (kg):"))
height = float(input("Enter your height (cm):"))
gender = input("Gender:")
age = float(input("Select your age:"))
bmi = round(weight/((height/100)**2),2)
if bmi < 18.5:
    bodytype="Underweight"
elif 18.5 <= bmi < 25:
    bodytype= "Normal Weight"
elif 25 <= bmi < 30:
    bodytype= "Overweight"
else:
    bodytype= "Obese"
print(bodytype)
if gender.lower()=="male":
    bmr=88.362+(13.397*weight)+(4.799*height)-(5.677*age)
elif gender.lower()=="Female":
    bmr=447.593+(9.247*weight)+(3.098*height)-(4.330*age)
else:
    print("Fuck Off")
print(bmr)
activity_level_options = ['Sedentary', 'Lightly_Active', 'Moderately_Active', 'Very_Active']
activity_level = input(["['Sedentary', 'Lightly_Active', 'Moderately_Active', 'Very_Active']"])
multiplier = {
    'sedentary': 1.2,
    'lightly_active': 1.375,
    'moderately_active': 1.55,
    'very_active': 1.725
    }
tdee = bmr*multiplier[activity_level.lower()]  
print(tdee)

Enter your weight (kg): 65
Enter your height (cm): 165
Gender: male
Select your age: 19


Normal Weight
1643.139


["['Sedentary', 'Lightly_Active', 'Moderately_Active', 'Very_Active']"] Lightly_Active


2259.316125


In [13]:
data = pd.read_csv("diertary_recomendation.csv")
data.head()

,food,calorie,protein,carbs,fat,Goals,Preferences,Restrictions
0,Chicken Breast,165,31.0,0.0,3.6,Weight loss,Protein-Rich Foods,Lactose intolerant
1,Salmon,206,22.0,0.0,13.0,Muscle gain,Protein-Rich Foods,NaN
2,Brown Rice,215,5.0,45.0,1.6,General health,Whole Grains,Gluten-free
3,Broccoli,55,3.7,11.0,0.6,Weight loss,Vegetables,Lactose intolerant
4,Almonds,7,0.3,0.6,0.6,Weight loss,Protein-Rich Foods,NaN


In [17]:
l1 = data['Goals'].unique().tolist()
l2 = data['Preferences'].unique().tolist()
l3 = data['Restrictions'].unique().tolist()
print(l1)
print(l2)
print(l3)

['Weight loss', 'Muscle gain', 'General health', 'Indulgence']
['Protein-Rich Foods', 'Whole Grains', 'Vegetables', 'Dairy', 'Fruits', nan, 'Vegetarian', 'Nuts', 'Spices', 'Fats', 'Sweets', 'Beverages', 'Seafood', 'Snacks', 'Condiments', 'Street Food', 'Indian', 'South Indian', 'Fusion', 'North Indian', 'Tibetan', 'Indo-Chinese', 'Gujarati', 'Rajasthani']
['Lactose intolerant', nan, 'Gluten-free', 'Vegetarian', 'Caffeine sensitive']


In [19]:
print(data.shape[0])

891


In [20]:
data.drop_duplicates(inplace=True)
data.reset_index(drop=True, inplace=True)
print(data.shape[0])

750


In [24]:
data['Restrictions'].fillna('No Restriction', inplace=True)
data.head()

,food,calorie,protein,carbs,fat,Goals,Preferences,Restrictions
0,Chicken Breast,165,31.0,0.0,3.6,Weight loss,Protein-Rich Foods,Lactose intolerant
1,Salmon,206,22.0,0.0,13.0,Muscle gain,Protein-Rich Foods,No Restriction
2,Brown Rice,215,5.0,45.0,1.6,General health,Whole Grains,Gluten-free
3,Broccoli,55,3.7,11.0,0.6,Weight loss,Vegetables,Lactose intolerant
4,Almonds,7,0.3,0.6,0.6,Weight loss,Protein-Rich Foods,No Restriction


In [26]:
data.dropna()

,food,calorie,protein,carbs,fat,Goals,Preferences,Restrictions
0,Chicken Breast,165,31.0,0.0,3.6,Weight loss,Protein-Rich Foods,Lactose intolerant
1,Salmon,206,22.0,0.0,13.0,Muscle gain,Protein-Rich Foods,No Restriction
2,Brown Rice,215,5.0,45.0,1.6,General health,Whole Grains,Gluten-free
3,Broccoli,55,3.7,11.0,0.6,Weight loss,Vegetables,Lactose intolerant
4,Almonds,7,0.3,0.6,0.6,Weight loss,Protein-Rich Foods,No Restriction
...,...,...,...,...,...,...,...,...
745,Cham Cham Cupcakes,180,3.0,20.0,8.0,Indulgence,Sweets,No Restriction
746,Mysore Pak Ice Cream,250,5.0,30.0,12.0,Indulgence,Sweets,No Restriction
747,Gajar Ki Kheer,150,3.0,20.0,6.0,Indulgence,Sweets,No Restriction
748,Malai Sandwich,200,4.0,25.0,8.0,Indulgence,Sweets,No Restriction


In [37]:
def recommend_food(goals, preferences, restrictions, filtered_df):
    # Define weights for goals, preferences, and restrictions
    goal_weight = 2
    preference_weight = 1
    restriction_weight = -1
    
    filtered_df = filtered_df.copy()
    # Calculate scores for each food item
    filtered_df.loc[:, 'Score'] = (
        filtered_df['Goals'].apply(lambda x: goal_weight if x in goals else 0) +
        filtered_df['Preferences'].apply(lambda x: preference_weight if any(pref in preferences for pref in x.split(',')) else 0) +
        filtered_df['Restrictions'].apply(lambda x: restriction_weight if any(rest in restrictions for rest in x.split(',')) else 0)
    )

    # Sort DataFrame based on scores
    recommended_df = filtered_df.sort_values(by='Score', ascending=False).head(1)

    return recommended_df.to_dict(orient='records')[0]

In [38]:
goals = ["Weight loss"]
preferences = ['Beverages', 'Seafood', 'Snacks']
restrictions = ["No Restriction"]
filtered_data = data[data['Goals'].isin(goals) & data['Preferences'].apply(lambda x: isinstance(x, str) and any(pref in preferences for pref in x.split(','))) & data['Restrictions'].apply(lambda x: isinstance(x, str) and any(rest in restrictions for rest in x.split(',')))]

recomdation = recommend_food(goals,preferences,restrictions,filtered_data)

In [39]:
print(recomdation)

{'food': 'Grilled Salmon Salad', 'calorie': 250, 'protein': 30.0, 'carbs': 15.0, 'fat': 12.0, 'Goals': 'Weight loss', 'Preferences': 'Seafood', 'Restrictions': 'No Restriction', 'Score': 2}


In [61]:
def recommend_food(goals, preferences, restrictions, filtered_data, body_goal):
    # Define macronutrient distribution based on body goals
    if body_goal == 'Weight Loss':
        protein_range = (1.2, 1.6)
    elif body_goal == 'Muscle Gain':
        protein_range = (1.6, 2.2)
    elif body_goal == 'Maintenance':
        protein_range = (1.2, 1.6)
    else:
        protein_range = (1.2, 1.6)  # Default for General Health

    fat_range = (0.2, 0.35)  # 20-35% of total daily calories

    # Create a deep copy to avoid SettingWithCopyWarning
    filtered_data = filtered_data.copy()

    # Calculate scores for each food item using .loc
    filtered_data.loc[:, 'Score'] = (
        filtered_data['Goals'].apply(lambda x: 1 if x == body_goal else 0) +
        filtered_data['Preferences'].apply(lambda x: 1 if isinstance(x, str) and any(pref in preferences for pref in x.split(',')) else 0) +
        filtered_data['Restrictions'].apply(lambda x: 1 if isinstance(x, str) and any(rest in restrictions for rest in x.split(',')) else 0)
    )

    # Filter data based on user inputs
    filtered_data = filtered_data[filtered_data['Goals'].apply(lambda x: x[0] == body_goal)]


    # Recommend food based on user inputs
    recommendation = filtered_data.sort_values(by='Score', ascending=False).head(1).to_dict(orient='records')[0]

    # Calculate macronutrient distribution based on body goals
    weight = st.number_input("Enter your weight (kg):")
    protein_range = (weight * protein_range[0], weight * protein_range[1])
    fat_range = (weight * fat_range[0], weight * fat_range[1])
    carb_range = (weight * 4 - protein_range[0] * 4 - fat_range[0] * 9, weight * 4 - protein_range[1] * 4 - fat_range[1] * 9)

    # Display macronutrient distribution ranges
    print("Macronutrient Distribution Ranges:")
    print(f"Protein: {protein_range[0]:.2f} - {protein_range[1]:.2f} grams")
    print(f"Fat: {fat_range[0]:.2f} - {fat_range[1]:.2f} grams")
    print(f"Carbohydrates: {carb_range[0]:.2f} - {carb_range[1]:.2f} grams")

In [62]:
goals = ["Weight loss"]
preferences = ['Beverages', 'Seafood', 'Snacks']
restrictions = ["No Restriction"]
body_goal=["Weight loss"]
filtered_data = data[data['Goals'].isin(goals) & data['Preferences'].apply(lambda x: isinstance(x, str) and any(pref in preferences for pref in x.split(','))) & data['Restrictions'].apply(lambda x: isinstance(x, str) and any(rest in restrictions for rest in x.split(',')))]

print("Filtered DataFrame based on user inputs:")
print(filtered_data)


Filtered DataFrame based on user inputs:
                     food  calorie  protein  carbs   fat        Goals  \
312  Grilled Salmon Salad      250     30.0   15.0  12.0  Weight loss   
330    Grilled Tuna Steak      180     25.0    1.0   8.0  Weight loss   
347        Baked Cod Fish      120     25.0    0.0   2.0  Weight loss   

    Preferences    Restrictions  
312     Seafood  No Restriction  
330     Seafood  No Restriction  
347     Seafood  No Restriction  


In [63]:
 recommend_food(goals, preferences, restrictions, filtered_data, body_goal)

IndexError: list index out of range

In [58]:
print(len(filtered_data['Goals']), len(body_goal))

3 3


In [45]:
print(filtered_data['Goals'])

312    Weight loss
330    Weight loss
347    Weight loss
Name: Goals, dtype: object


In [47]:
l=["Weight loss"]*len(filtered_data['Goals'])
print(l)

['Weight loss', 'Weight loss', 'Weight loss']


In [ ]:
weight = float(input("Enter your weight (kg):"))
height = float(input("Enter your height (cm):"))
gender = input("Gender:")
age = float(input("Select your age:"))

# Calculate BMI
bmi = round(weight / ((height / 100) ** 2), 2)

# Determine body type based on BMI
if bmi < 18.5:
    body_type = "Underweight"
elif 18.5 <= bmi < 25:
    body_type = "Normal Weight"
elif 25 <= bmi < 30:
    body_type = "Overweight"
else:
    body_type = "Obese"

print("Body Type:", body_type)

# Calculate BMR based on gender
if gender.lower() == "male":
    bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
elif gender.lower() == "female":
    bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)
else:
    print("Invalid gender")

print("Basal Metabolic Rate (BMR):", bmr)

# Get user's activity level
activity_level = input("Select your activity level: ['Sedentary', 'Lightly_Active', 'Moderately_Active', 'Very_Active'] ")

# Activity level multiplier
multiplier = {
    'sedentary': 1.2,
    'lightly_active': 1.375,
    'moderately_active': 1.55,
    'very_active': 1.725
}

# Calculate Total Daily Energy Expenditure (TDEE)
tdee = bmr * multiplier.get(activity_level.lower(), 1)

print("Total Daily Energy Expenditure (TDEE):", tdee)
